# Time Series Analysis: "The Final Project"

`End? No, the journey doesn't end here. Death is just another path. One that we all must take.
-J.R.R. Tolkien, The Return of the King`

---

## Libraries

In [1]:
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
import statsmodels.api as sm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from pandas.plotting import register_matplotlib_converters
from IPython.display import display
# from tsa_functions import *

register_matplotlib_converters()
sns.set_style('darkgrid')

np.set_printoptions(precision=4)
pd.set_option('precision', 4)


In [2]:
from tsa_tools import *

---

## M5 Forecasting

For this "final project", we will be forecasting the <b><u>level 9</b></u> series (unit sales of all products, aggregated for each store and department).

Load `sales_train_evaluation.csv` and use observations from `d_1 to d_1913` for training and `d_1914 to d_1941` for testing.

In [3]:
df_calendar = pd.read_csv('../data/m5/calendar.csv')
df_sales = pd.read_csv('../data/m5/sales_train_evaluation.csv')
df_weights = pd.read_csv('../data/m5/weights_validation.csv')
# display(df_calendar, df_sales, df_weights)

In [4]:
train_df = (df_sales.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)[:1941]).iloc[:-28])
test_df = (df_sales.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)[:1941]).iloc[-28:])
# display(train_df, test_df)

In [5]:
levels = {
    1: None,
    2: "state_id",
    3: "store_id",
    4: "cat_id",
    5: "dept_id",
    6: ["state_id", "cat_id"],
    7: ["state_id", "dept_id"],
    8: ["store_id", "cat_id"],
    9: ["store_id", "dept_id"],
    10: "item_id",
    11: ["state_id", "item_id"],
    12: ["store_id", "item_id"]
}


---

## Part 1. Baseline Methods (10 pts.)

### Q1. (10 pts.)

Extract all level 9 series from the dataset.

For each series, generate a 28-step forecast using the methods enumerated below and calculate the `RMSSE` against the test set:

1. `Naive`


2. `Seasonal Naive`


3. `SES`


4. `Holt's Linear`


5. `Additive Holt-Winters`

Summarize the metrics in a dataframe and print it.

In [6]:
methods = {
    "Naive": BaseFuncModel(naivef),
    "Seasonal Naive": BaseFuncModel(snaivef, m=7),
    "SES": StatsModelsWrapper(ETSModel, trend=None, seasonal=None),
    "Holt's Linear": StatsModelsWrapper(ETSModel, trend='add', seasonal=None),
    "Additive Holt-Winters": StatsModelsWrapper(
        ETSModel, seasonal_periods=7, trend='add', seasonal='add'),
}

train_df_9 = timeSeriesFiltering(
    train_df.sum(axis='columns', level=levels[9]), lower=10)
test_df_9 = test_df.sum(axis='columns', level=levels[9])
weights_df_9 = (df_weights
                .loc[df_weights['Level_id'] == 'Level9']
                .set_index(['Agg_Level_1', 'Agg_Level_2'])[['Weight']])

In [7]:
res = {}
for method, model in methods.items():
    forecast_df_9 = pd.DataFrame(
        {label: model.fit(content).forecast(28)
        for label, content in train_df_9.items()}
        )
    res[method] = rateMyForecast(train_df_9, test_df_9, forecast_df_9)['RMSSE']

In [8]:
pd.set_option('display.max_rows', None)
df_res_9_base = pd.DataFrame(res)
df_res_9_base.index = pd.MultiIndex.from_tuples(df_res_9_base.index)
df_res_9_base

Naive  Seasonal Naive     SES  Holt's Linear  \
CA_1 HOBBIES_1    1.4583          0.7619  0.8819         0.8810   
     HOBBIES_2    1.9340          1.1429  0.8831         0.8748   
     HOUSEHOLD_1  2.1052          0.5172  1.1514         1.1636   
     HOUSEHOLD_2  2.2997          0.5228  1.2208         1.1789   
     FOODS_1      0.9319          0.7312  0.9179         0.9362   
     FOODS_2      2.0535          0.8269  2.0534         2.3136   
     FOODS_3      1.7113          0.4944  1.0825         1.0185   
CA_2 HOBBIES_1    1.2053          0.7055  1.1179         1.1180   
     HOBBIES_2    1.3546          1.4893  1.3887         1.3790   
     HOUSEHOLD_1  2.0287          0.6373  1.3960         1.3853   
     HOUSEHOLD_2  1.9340          0.5920  1.4625         1.4534   
     FOODS_1      1.2557          1.1311  1.3875         1.3786   
     FOODS_2      4.2936          1.4906  2.3041         2.2415   
     FOODS_3      2.5830          0.6046  1.4502         1.4114   
CA_3 HOBBIES_1    0.9370          0.6647  0.6999         0.7018   
     HOBBIES_2    1.1984          1.6617  1.5530         1.5433   
     HOUSEHOLD_1  2.6217          0.7564  1.1273         1.1536   
     HOUSEHOLD_2  2.1039          0.8808  1.4258         1.4169   
     FOODS_1      1.1674          1.5874  1.3966         1.3892   
     FOODS_2      1.8816          0.7316  1.7271         1.7429   
     FOODS_3      1.5710          0.4353  0.8908         1.5877   
CA_4 HOBBIES_1    1.4556          1.1008  0.9956         0.9902   
     HOBBIES_2    2.1142          1.8563  1.8756         1.8751   
     HOUSEHOLD_1  3.4088          1.4737  1.2447         1.2169   
     HOUSEHOLD_2  1.3534          0.9567  1.0520         1.0386   
     FOODS_1      1.0167          1.5664  1.3857         1.3656   
     FOODS_2      1.2989          0.7998  1.1609         1.1533   
     FOODS_3      1.4144          0.6385  0.9469         0.9431   
TX_1 HOBBIES_1    1.6331          0.9169  0.9061         0.9046   
     HOBBIES_2    1.8823          1.6475  1.4296         1.4159   
     HOUSEHOLD_1  1.8399          0.8196  1.1676         1.1608   
     HOUSEHOLD_2  1.2045          1.2523  1.0488         1.0328   
     FOODS_1      1.3344          1.7189  1.5566         1.5460   
     FOODS_2      1.1906          0.9520  0.8562         0.8561   
     FOODS_3      1.1678          0.8416  0.9161         0.8855   
TX_2 HOBBIES_1    0.7979          0.8792  0.8789         0.8742   
     HOBBIES_2    0.7499          1.2548  0.9735         0.9685   
     HOUSEHOLD_1  0.8662          0.8529  0.8683         0.8712   
     HOUSEHOLD_2  1.3706          0.6559  0.9314         0.9346   
     FOODS_1      1.4774          1.2816  1.2919         1.2839   
     FOODS_2      0.9424          0.6834  0.7532         0.7719   
     FOODS_3      0.8228          0.6226  0.7499         0.7481   
TX_3 HOBBIES_1    1.5309          0.9025  0.9404         0.9370   
     HOBBIES_2    1.4339          1.7212  1.1186         1.1152   
     HOUSEHOLD_1  1.0738          1.0794  1.0248         1.0064   
     HOUSEHOLD_2  0.8605          1.3137  1.0128         0.9889   
     FOODS_1      2.7332          2.2206  2.1542         2.1356   
     FOODS_2      1.3682          1.4195  1.4254         1.4277   
     FOODS_3      0.9672          1.1046  1.0692         1.0493   
WI_1 HOBBIES_1    1.3685          0.5084  0.7533         0.7533   
     HOBBIES_2    1.2701          1.5892  1.2413         1.2333   
     HOUSEHOLD_1  1.7201          0.6488  1.0982         1.0990   
     HOUSEHOLD_2  1.7301          0.7509  1.1014         1.0997   
     FOODS_1      0.9843          0.8454  1.0438         1.0400   
     FOODS_2      2.1064          1.2493  1.6349         1.6291   
     FOODS_3      1.6655          0.5440  1.1554         1.1465   
WI_2 HOBBIES_1    1.3672          0.8269  0.7654         0.7546   
     HOBBIES_2    1.1960          1.5164  1.2308         1.2258   
     HOUSEHOLD_1  1.3142          1.0984  1.3773         1.3079   

---

## Part 2. LightGBM (30 pts.)

### Q2. (10 pts.)

For all series, use an un-tuned `LightGBM` with 56-day lookback that uses a one-step recursive forecasting strategy to generate a 28-step forecast.

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [9]:
# Your code here

### Q3. (10 pts.)

For all series, use an un-tuned `LightGBM` with 56-day lookback that uses a direct forecasting strategy to generate a 28-step forecast.

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [14]:
model = MultiOutputRegressor(
    lgb.LGBMRegressor(random_state=1),
    n_jobs=1)  # Model: direct-forecasting
pred = {}

for col in train_df_9:
    X_train, X_test, y_train, y_test = TimeseriesGenerator(
        X=train_df_9[col],
        y=test_df_9[col],
        w=56,
        h=28)
    model.fit(X_train, y_train)
    pred[col] = model.predict(X_test).squeeze()

In [16]:
df_morlgb_9_base = pd.DataFrame(pred, index=test_df_9.index)
res["MultiOutputRegressor(LGBMRegressor)"] = rateMyForecast(
    test_df_9, )

CA_1                                                        \
           HOBBIES_1 HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2   FOODS_1   FOODS_2   
date                                                                         
2016-04-25  471.7198   45.1463    771.9727    193.6524  276.0039  535.7883   
2016-04-26  402.9137   42.7726    647.3247    176.7904  251.2834  419.5163   
2016-04-27  375.4826   38.6349    572.8614    183.9880  291.8602  348.5578   
2016-04-28  425.8438   41.1084    615.2042    174.4992  309.6623  328.3124   
2016-04-29  454.9795   48.8006    723.3772    208.3274  313.3161  438.4479   
2016-04-30  612.5016   55.8828   1017.1668    264.9571  376.4439  532.8147   
2016-05-01  545.9712   54.1655   1101.3459    268.7070  322.8507  611.4983   
2016-05-02  479.6887   40.6675    781.4924    191.6617  281.1408  521.5170   
2016-05-03  456.6566   42.9033    652.4823    187.6411  278.3609  492.8575   
2016-05-04  434.7710   40.7363    642.2365    185.0418  251.8087  514.1009   
2016-05-05  431.0660   39.9970    673.3987    206.2273  290.5343  472.3082   
2016-05-06  518.6863   42.5698    789.7189    232.7969  365.0822  629.4371   
2016-05-07  590.3889   53.0582   1028.5885    289.3132  439.9348  752.4259   
2016-05-08  559.9750   43.1924    916.8236    262.7275  372.0284  735.0582   
2016-05-09  501.0072   42.4271    722.5863    185.3636  293.5527  556.1328   
2016-05-10  478.5058   41.2686    624.9683    190.9131  269.1880  462.0862   
2016-05-11  417.7204   48.2054    607.1876    169.9432  329.6042  432.4742   
2016-05-12  437.7128   46.1874    636.0689    203.1090  314.2414  393.4193   
2016-05-13  476.0144   40.6487    763.8531    208.4195  375.6040  490.2832   
2016-05-14  580.7841   53.5199   1035.5844    283.2642  466.0117  589.0536   
2016-05-15  536.5159   45.7763   1010.5037    283.6756  354.3511  661.1707   
2016-05-16  499.4256   39.0533    779.4293    182.8448  340.1006  487.0805   
2016-05-17  442.6632   28.0561    637.0052    176.8414  349.2529  447.1419   
2016-05-18  420.5836   35.0444    593.2674    191.6277  297.9451  432.2118   
2016-05-19  474.8373   35.3375    644.8794    199.6080  322.9043  375.8990   
2016-05-20  443.1410   46.9198    770.0047    214.1130  352.7802  451.7857   
2016-05-21  550.5156   51.2719   1004.6719    285.3019  388.7694  564.3715   
2016-05-22  531.7143   57.5252   1075.9692    288.4098  356.1310  598.0216   

                           CA_2                        ...      WI_2  \
              FOODS_3 HOBBIES_1 HOBBIES_2 HOUSEHOLD_1  ...   FOODS_1   
date                                                   ...             
2016-04-25  1904.3421  315.1318   42.4370    658.0965  ...  338.4639   
2016-04-26  1901.9920  339.6029   38.6299    575.0550  ...  313.9521   
2016-04-27  1722.6674  293.4951   42.8916    638.9592  ...  302.0617   
2016-04-28  1702.9085  333.0378   37.0392    611.7359  ...  259.6962   
2016-04-29  2230.8995  360.8896   37.9370    830.4953  ...  324.4630   
2016-04-30  2696.4826  451.2706   42.4050   1071.8683  ...  365.2638   
2016-05-01  2870.8932  441.8988   49.3652   1117.7769  ...  351.6498   
2016-05-02  2013.1358  322.5711   33.7356    659.7563  ...  360.4940   
2016-05-03  1882.3660  302.0023   36.9672    605.0382  ...  316.5374   
2016-05-04  1835.1287  309.8994   55.5876    652.3614  ...  328.6509   
2016-05-05  1829.9592  311.8219   34.6853    633.6771  ...  315.2633   
2016-05-06  2256.2421  434.5299   40.1232    840.6947  ...  307.6017   
2016-05-07  2786.5129  492.9049   47.1770   1097.8663  ...  368.1233   
2016-05-08  2939.3219  430.0286   38.0759   1142.4721  ...  328.9731   
2016-05-09  2140.5727  282.5624   33.3249    614.7498  ...  330.3418   
2016-05-10  1843.8994  314.1087   40.7778    615.9756  ...  357.6347   
2016-05-11  1811.0421  292.5572   39.3048    594.6787  ...  282.7229   
2016-05-12  1887.7493  302.1852   35.1498    608.6523  ...  287.4665   
2016-05-13  2087.8597  369.2011   55.8436    802.7517  ...  364.7806   
2016-05-14  2

In [ ]:
pipe.transform(train_df_9.iloc[:, 0])

transforming date
2011-01-29    528.0
2011-01-30    489.0
2011-01-31    409.0
2011-02-01    383.0
2011-02-02    263.0
2011-02-03    453.0
2011-02-04    339.0
2011-02-05    750.0
2011-02-06    425.0
2011-02-07    375.0
2011-02-08    485.0
2011-02-09    387.0
2011-02-10    444.0
2011-02-11    443.0
2011-02-12    674.0
2011-02-13    446.0
2011-02-14    376.0
2011-02-15    322.0
2011-02-16    266.0
2011-02-17    262.0
2011-02-18    288.0
2011-02-19    780.0
2011-02-20    472.0
2011-02-21    606.0
2011-02-22    444.0
2011-02-23    398.0
2011-02-24    419.0
2011-02-25    392.0
2011-02-26    598.0
2011-02-27    196.0
2011-02-28    169.0
2011-03-01    217.0
2011-03-02    187.0
2011-03-03    163.0
2011-03-04    175.0
2011-03-05    279.0
2011-03-06    724.0
2011-03-07    452.0
2011-03-08    455.0
2011-03-09    346.0
2011-03-10    315.0
2011-03-11    324.0
2011-03-12    626.0
2011-03-13    391.0
2011-03-14    348.0
2011-03-15    322.0
2011-03-16    359.0
2011-03-17    359.0
2011-03-18    283.0
20

array([[528., 489., 409., ..., 288., 508., 424.],
       [489., 409., 383., ..., 508., 424., 573.],
       [409., 383., 263., ..., 424., 573., 517.],
       ...,
       [564., 608., 498., ..., 413., 445., 407.],
       [608., 498., 381., ..., 445., 407., 557.],
       [498., 381., 589., ..., 407., 557., 338.]])

In [ ]:
tranny.fit(pipe.transform(train_df_9.iloc[:, 0]), train_df_9.iloc[:, 0])

transforming date
2011-01-29    528.0
2011-01-30    489.0
2011-01-31    409.0
2011-02-01    383.0
2011-02-02    263.0
2011-02-03    453.0
2011-02-04    339.0
2011-02-05    750.0
2011-02-06    425.0
2011-02-07    375.0
2011-02-08    485.0
2011-02-09    387.0
2011-02-10    444.0
2011-02-11    443.0
2011-02-12    674.0
2011-02-13    446.0
2011-02-14    376.0
2011-02-15    322.0
2011-02-16    266.0
2011-02-17    262.0
2011-02-18    288.0
2011-02-19    780.0
2011-02-20    472.0
2011-02-21    606.0
2011-02-22    444.0
2011-02-23    398.0
2011-02-24    419.0
2011-02-25    392.0
2011-02-26    598.0
2011-02-27    196.0
2011-02-28    169.0
2011-03-01    217.0
2011-03-02    187.0
2011-03-03    163.0
2011-03-04    175.0
2011-03-05    279.0
2011-03-06    724.0
2011-03-07    452.0
2011-03-08    455.0
2011-03-09    346.0
2011-03-10    315.0
2011-03-11    324.0
2011-03-12    626.0
2011-03-13    391.0
2011-03-14    348.0
2011-03-15    322.0
2011-03-16    359.0
2011-03-17    359.0
2011-03-18    283.0
20

ValueError: Found array with dim 3. Estimator expected <= 2.

### Q4. (10 pts.)

For all series, generate a 28-step forecast by combining the forecasts generated by the models in Q2 and Q3 (i.e. simple averaging).

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [ ]:
# Your code here

---

## Part 3. WRMSSE (10 pts.)

### Q5.  (10 pts.)

Calculate the `WRMSSE` for the all the methods described above. The weights can be found in `weights_validation.csv`.

For reference, the M5 benchmarks have the following `WRMSSE` scores at level 9:

- `Naive` = <b>1.764</b>


- `S.Naive` = <b>0.888</b>


- `ES_bu` = <b>0.728</b>

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

In [ ]:
(df_res_9_base.rename_axis(['Agg_Level_1', 'Agg_Level_2'])
 .multiply(weights_df_9.squeeze(), axis=0).sum())

Naive                    1.6287
Seasonal Naive           0.9300
SES                      1.2075
Holt's Linear            1.2367
Additive Holt-Winters    0.8549
dtype: float64

---

## Part 4. Middle-Out Method (30 pts.)

### Q6. Bottom-Up (15 pts.)

Using your forecasts from the best performing method in Q5, use the bottom-up method described in [FPP3](https://otexts.com/fpp3/single-level.html) to generate forecasts for levels 1 to 8.

Calculate the `WRMSSE` for levels 1 to 8 against the test set, then summarize the metrics in a dataframe and print it.

For reference, you can find the benchmark `WRMSSE` scores in the `The M5 Accuracy competition: Results, findings and conclusions` paper.

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

In [ ]:
# Your code here

### Q7. Top-Down  (15 pts.)

Using your forecasts from the best performing method in Q5, use the top-down method with `average historical proportions` described in [FPP3](https://otexts.com/fpp3/single-level.html) to generate forecasts for levels 10 to 12.

Calculate the `WRMSSE` for levels 10 to 12  against the test set, then summarize the metrics in a dataframe and print it.

For reference, you can find the benchmark `WRMSSE` scores in the `The M5 Accuracy competition: Results, findings and conclusions` paper.

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

In [ ]:
# Your code here